# Data Process

This notebook processes simulation data from `raw_sim_data` in the same way as `Asymm_v_T0_121323.xlsx`.

## Notebook Configuration

In [ ]:
# import packages
import os
from pathlib import Path

import pandas as pd
import numpy as np
import scipy as sp

from rich import print
from tqdm import tqdm